In [41]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query = """
(SELECT 
*, "Regular Season" as Category
FROM `perceptive-ivy-290216.nba_api_individual_matchups.all_time_reg_season` 
)
UNION ALL
(SELECT 
*, "Playoffs" as Category
FROM `perceptive-ivy-290216.nba_api_individual_matchups.all_time_playoffs`
)
"""
project_id = "perceptive-ivy-290216"
matchups_totals = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [4]:
query = """
SELECT PLAYER_NAME, TEAM_ABBREVIATION, SEASON FROM `perceptive-ivy-290216.player_team_info.player_bios_regs_totals`
"""
project_id = "perceptive-ivy-290216"
player_team = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [22]:
query = """
SELECT GAME_ID, GAME_DATE, MATCHUP	
FROM `perceptive-ivy-290216.nba_game_player_team_ids.game_ids_reg_season`
WHERE MATCHUP LIKE "%@%"
UNION ALL
SELECT GAME_ID,GAME_DATE, MATCHUP
FROM `perceptive-ivy-290216.nba_game_player_team_ids.game_ids_playoffs`
WHERE MATCHUP LIKE "%@%"
"""
project_id = "perceptive-ivy-290216"
game_ids = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [11]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#1d428a'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#6a00ed'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#85714d'
    elif team =="MIN":
        return '#236192'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#965ddd'                  
    
player_team['color'] = player_team['TEAM_ABBREVIATION'].apply(team_color)

In [15]:
#REGULAR SEASON

In [16]:
#TOTALS

In [7]:
year="2023-24"

In [93]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_totals_year=matchups_totals[matchups_totals["Season"]==year].merge(player_team[player_team["SEASON"]==year], left_on=["PLAYER_NAME","Season"], right_on=["PLAYER_NAME","SEASON"])
matchup_totals_year=matchup_totals_year.merge(game_ids, left_on=["GAME_ID"], right_on=["GAME_ID"])
player_color_dict_shot_dist=dict(zip(matchup_totals_year['PLAYER_NAME'], matchup_totals_year['color']))
team_color_dict_shot_dist=dict(zip(matchup_totals_year['TEAM_ABBREVIATION'], matchup_totals_year['color']))
matchup_totals_year.tail()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,POSITION,MATCHUP_NAME,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,helpFieldGoalsPercentage,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFieldGoalsPercentage,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutes,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupThreePointersPercentage,matchupTurnovers,partialPossessions,percentageDefenderTotalTime,percentageOffensiveTotalTime,percentageTotalTimeBothOn,playerPoints,shootingFouls,switchesOn,teamPoints,Season,Category,TEAM_ABBREVIATION,SEASON,color,GAME_DATE,MATCHUP
244230,0022300712,Denver Nuggets,Portland Trail Blazers,Portland Trail Blazers,Toumani Camara,,Peyton Watson,0,0,0,0.0,0,0,3,2,0.667,0,0,1:19,78,0,1,1,1.000,1,4.8,0.108,0.146,0.164,5,0,0,11,2023-24,Regular Season,POR,2023-24,#C8102E,2024-02-04,POR @ DEN
244231,0022300348,Portland Trail Blazers,Golden State Warriors,Portland Trail Blazers,Jerami Grant,F,Dario Saric,0,0,0,0.0,2,0,4,2,0.500,0,0,1:01,60,0,3,1,0.333,1,3.9,0.186,0.068,0.212,5,0,0,14,2023-24,Regular Season,POR,2023-24,#C8102E,2023-12-17,GSW @ POR
244232,0022300986,Chicago Bulls,Portland Trail Blazers,Portland Trail Blazers,Anfernee Simons,G,Onuralp Bitim,0,0,0,0.0,0,0,3,1,0.333,2,2,0:40,40,0,3,1,0.333,0,5.1,0.069,0.045,0.087,5,1,0,10,2023-24,Regular Season,POR,2023-24,#C8102E,2024-03-18,POR @ CHI
244233,0022301183,Portland Trail Blazers,Houston Rockets,Portland Trail Blazers,Dalano Banton,,Dillon Brooks,0,0,0,0.0,1,0,3,1,0.333,2,2,1:23,83,0,3,1,0.333,0,8.5,0.116,0.092,0.165,5,0,0,11,2023-24,Regular Season,POR,2023-24,#C8102E,2024-04-12,HOU @ POR
244234,0022300881,Minnesota Timberwolves,Portland Trail Blazers,Portland Trail Blazers,Dalano Banton,F,Anthony Edwards,0,0,0,0.0,0,0,5,3,0.600,0,0,3:30,209,0,1,0,0.000,0,14.7,0.238,0.249,0.318,6,0,0,13,2023-24,Regular Season,POR,2023-24,#C8102E,2024-03-04,POR @ MIN


In [ ]:
#By Individual Games

In [27]:
game_id="0022300712"

In [28]:
#Filter to regular season
matchup_season=matchup_totals_year[(matchup_totals_year["Category"]=='Regular Season')&(matchup_totals_year["GAME_ID"]==game_id)]
#Create a player and team dict for color for future coloring

In [65]:
matchup=matchup_season['MATCHUP'].iloc[0]
date=matchup_season['GAME_DATE'].iloc[0]

In [66]:
category="matchupFreeThrowsAttempted"

# category=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [67]:
fig_matchup_season = px.bar(
  matchup_season[matchup_season[category]!=0].sort_values(by=category, ascending=False),
  x="PLAYER_NAME", 
  y=category,
  color="PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Player {} For The {} Game on {}</b>".format(category,matchup,date),
  hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
       'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
       'Category', 'SEASON', ], 
  height=800, 
  width=1200,
  )
fig_matchup_season.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season

In [ ]:
save_year=2024

In [ ]:
fig_matchup_season.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Game Level/{}/Player {} For The {} Game on {}".format(save_year,category,matchup,date),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [ ]:
category_multi=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_multi:
       fig_matchup_season_multi = px.bar(
       matchup_season[matchup_season[i]!=0].sort_values(by=i, ascending=False),
       x="PLAYER_NAME", 
       y=i,
       color="PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Player {} For The {} Game on {}</b>".format(i,matchup,date),
       hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
              'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
              'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
              'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
              'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
              'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
              'matchupPotentialAssists', 'matchupThreePointersAttempted',
              'matchupThreePointersMade', 'matchupThreePointersPercentage',
              'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
              'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
              'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
              'Category', 'SEASON', ], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_multi
       fig_matchup_season_multi.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Game Level/{}/Player {} For The {} Game on {}".format(save_year,i,matchup,date),full_html=False, include_plotlyjs='cdn')

In [ ]:
# For Total Season

In [72]:
matchup_season_full=matchup_totals_year[(matchup_totals_year["Category"]=='Regular Season')]

In [83]:
matchup_season_full_totals=pd.DataFrame(matchup_season_full.groupby(by=['PLAYER_NAME',"SEASON"])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_full_totals["matchupFieldGoalsPercentage"]=matchup_season_full_totals["matchupFieldGoalsMade"]/matchup_season_full_totals["matchupFieldGoalsAttempted"]
matchup_season_full_totals["matchupThreePointersPercentage"]=matchup_season_full_totals["matchupThreePointersMade"]/matchup_season_full_totals["matchupThreePointersAttempted"]
matchup_season_full_totals["matchupFreeThrowsPercentage"]=matchup_season_full_totals["matchupFreeThrowsMade"]/matchup_season_full_totals["matchupFreeThrowsAttempted"]
matchup_season_full_totals=matchup_season_full_totals.fillna(0)

In [82]:
matchup_season_full_totals.head()

,PLAYER_NAME,SEASON,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,shootingFouls,switchesOn,teamPoints,matchupFieldGoalsPercentage,matchupThreePointersPercentage,matchupFreeThrowsPercentage
0,A.J. Lawson,2023-24,0,0,0,16,4,130,60,18,11,6753,0,45,12,12,654.6,143,9,0,694,0.461538,0.266667,0.611111
1,AJ Green,2023-24,0,0,0,29,2,201,86,11,10,13350,0,170,69,9,1186.8,251,4,0,1346,0.427861,0.405882,0.909091
2,AJ Griffin,2023-24,0,0,0,6,3,66,20,0,0,3904,0,38,11,7,347.4,51,0,0,369,0.30303,0.289474,NaN
3,Aaron Gordon,2023-24,0,0,0,250,40,827,448,212,141,55090,0,125,35,105,4635.5,1072,111,0,5785,0.541717,0.28,0.665094
4,Aaron Holiday,2023-24,0,0,0,138,18,432,194,44,41,27321,0,213,83,58,2506.8,512,17,0,2786,0.449074,0.389671,0.931818


In [85]:
category_full_season="matchupFreeThrowsAttempted"

# category=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [94]:
fig_matchup_season_full = px.bar(
  matchup_season_full_totals[matchup_season_full_totals[category_full_season]!=0].sort_values(by=category_full_season, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_full_season,
  color="PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Player {} For The {} Season</b>".format(category_full_season,year),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
  height=800, 
  width=1200,
  )
fig_matchup_season_full.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season_full

In [ ]:
fig_matchup_season_full.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Defender/{}/Player {} For The {} Season".format(save_year,category_full_season,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Metric Plot

In [ ]:
category_full_season=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_full_season:
       fig_matchup_season_full_multi = px.bar(
       matchup_season_full_totals[matchup_season_full_totals[category_full_season]!=0].sort_values(by=category_full_season, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_full_season,
       color="PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Player {} For The {} Season</b>".format(category_full_season,year),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_full_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_full.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Defender/{}/Player {} For The {} Season".format(save_year,i,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#For All Time 

In [108]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_season_all=matchups_totals[(matchups_totals["Category"]=='Regular Season')]
matchup_season_all_totals=pd.DataFrame(matchup_season_all.groupby(by=['PLAYER_NAME'])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_all_totals["matchupFieldGoalsPercentage"]=matchup_season_all_totals["matchupFieldGoalsMade"]/matchup_season_all_totals["matchupFieldGoalsAttempted"]
matchup_season_all_totals["matchupThreePointersPercentage"]=matchup_season_all_totals["matchupThreePointersMade"]/matchup_season_all_totals["matchupThreePointersAttempted"]
matchup_season_all_totals["matchupFreeThrowsPercentage"]=matchup_season_all_totals["matchupFreeThrowsMade"]/matchup_season_all_totals["matchupFreeThrowsAttempted"]
matchup_season_all_totals=matchup_season_all_totals.fillna(0)

In [109]:
category_all="matchupFreeThrowsAttempted"

# category_all=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [110]:
fig_matchup_all = px.bar(
  matchup_season_all_totals[matchup_season_all_totals[category_all]!=0].sort_values(by=category_all, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_all,
  template='plotly_white',
  title="<b>Regular Season All Time Player {}</b>".format(category_all),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
  height=800, 
  width=1200,
  )
fig_matchup_all.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_all.update_traces(marker_color='#0047ab')
fig_matchup_all

In [ ]:
fig_matchup_all.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/All Time Defender/Regular Season All Time Player {}".format(category_all),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Plots

In [ ]:
category_all=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_all:
       fig_matchup_all_multi = px.bar(
       matchup_season_all_totals[matchup_season_all_totals[category_all]!=0].sort_values(by=category_all, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_all,
       template='plotly_white',
       title="<b>Regular Season All Time Player {}</b>".format(category_all),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
       height=800, 
       width=1200,
       )
       fig_matchup_all_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_all_multi.update_traces(marker_color='#0047ab')
       fig_matchup_all_multi.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/All Time Defender/Regular Season All Time Player {}".format(category_all),full_html=False, include_plotlyjs='cdn')

In [ ]:
#PLAYOFFS

In [ ]:
#TOTALS

In [111]:
year_playoffs="2023-24"

In [112]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_totals_year_playoffs=matchups_totals[matchups_totals["Season"]==year].merge(player_team[player_team["SEASON"]==year], left_on=["PLAYER_NAME","Season"], right_on=["PLAYER_NAME","SEASON"])
matchup_totals_year_playoffs=matchup_totals_year_playoffs.merge(game_ids, left_on=["GAME_ID"], right_on=["GAME_ID"])
matchup_totals_year_playoffs.tail()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,POSITION,MATCHUP_NAME,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,helpFieldGoalsPercentage,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFieldGoalsPercentage,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutes,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupThreePointersPercentage,matchupTurnovers,partialPossessions,percentageDefenderTotalTime,percentageOffensiveTotalTime,percentageTotalTimeBothOn,playerPoints,shootingFouls,switchesOn,teamPoints,Season,Category,TEAM_ABBREVIATION,SEASON,color,GAME_DATE,MATCHUP
244230,0022300712,Denver Nuggets,Portland Trail Blazers,Portland Trail Blazers,Toumani Camara,,Peyton Watson,0,0,0,0.0,0,0,3,2,0.667,0,0,1:19,78,0,1,1,1.000,1,4.8,0.108,0.146,0.164,5,0,0,11,2023-24,Regular Season,POR,2023-24,#C8102E,2024-02-04,POR @ DEN
244231,0022300348,Portland Trail Blazers,Golden State Warriors,Portland Trail Blazers,Jerami Grant,F,Dario Saric,0,0,0,0.0,2,0,4,2,0.500,0,0,1:01,60,0,3,1,0.333,1,3.9,0.186,0.068,0.212,5,0,0,14,2023-24,Regular Season,POR,2023-24,#C8102E,2023-12-17,GSW @ POR
244232,0022300986,Chicago Bulls,Portland Trail Blazers,Portland Trail Blazers,Anfernee Simons,G,Onuralp Bitim,0,0,0,0.0,0,0,3,1,0.333,2,2,0:40,40,0,3,1,0.333,0,5.1,0.069,0.045,0.087,5,1,0,10,2023-24,Regular Season,POR,2023-24,#C8102E,2024-03-18,POR @ CHI
244233,0022301183,Portland Trail Blazers,Houston Rockets,Portland Trail Blazers,Dalano Banton,,Dillon Brooks,0,0,0,0.0,1,0,3,1,0.333,2,2,1:23,83,0,3,1,0.333,0,8.5,0.116,0.092,0.165,5,0,0,11,2023-24,Regular Season,POR,2023-24,#C8102E,2024-04-12,HOU @ POR
244234,0022300881,Minnesota Timberwolves,Portland Trail Blazers,Portland Trail Blazers,Dalano Banton,F,Anthony Edwards,0,0,0,0.0,0,0,5,3,0.600,0,0,3:30,209,0,1,0,0.000,0,14.7,0.238,0.249,0.318,6,0,0,13,2023-24,Regular Season,POR,2023-24,#C8102E,2024-03-04,POR @ MIN


In [ ]:
#By Individual Games

In [114]:
game_id_playoffs="0042300221"

In [115]:
#Filter to regular season
matchup_season_playoffs=matchup_totals_year_playoffs[(matchup_totals_year_playoffs["Category"]=='Playoffs')&(matchup_totals_year_playoffs["GAME_ID"]==game_id)]
#Create a player and team dict for color for future coloring

In [116]:
matchup_playoffs=matchup_season['MATCHUP'].iloc[0]
date_playoffs=matchup_season['GAME_DATE'].iloc[0]

In [117]:
category_playoffs="matchupFreeThrowsAttempted"

# category_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [ ]:
fig_matchup_season_playoffs = px.bar(
  matchup_season_playoffs[matchup_season_playoffs[category_playoffs]!=0].sort_values(by=category_playoffs, ascending=False),
  x="PLAYER_NAME", 
  y=category_playoffs,
  color="PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Player {} For The {} Game on {}</b>".format(category_playoffs,matchup_playoffs,date_playoffs),
  hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
       'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
       'Category', 'SEASON', ], 
  height=800, 
  width=1200,
  )
fig_matchup_season_playoffs.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season_playoffs

In [ ]:
fig_matchup_season_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Game Level/{}/Player {} For The {} Game on {}".format(save_year, category_playoffs,matchup_playoffs,date_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [ ]:
category_multi_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_multi_playoffs:
       fig_matchup_season_multi_playoffs = px.bar(
       matchup_season_playoffs[matchup_season_playoffs[i]!=0].sort_values(by=i, ascending=False),
       x="PLAYER_NAME", 
       y=i,
       color="PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Player {} For The {} Game on {}</b>".format(i,matchup,date),
       hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
              'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
              'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
              'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
              'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
              'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
              'matchupPotentialAssists', 'matchupThreePointersAttempted',
              'matchupThreePointersMade', 'matchupThreePointersPercentage',
              'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
              'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
              'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
              'Category', 'SEASON', ], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_multi_playoffs.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_multi_playoffs
       fig_matchup_season_multi_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Game Level/{}/Player {} For The {} Game on {}".format(save_year,i,matchup,date),full_html=False, include_plotlyjs='cdn')

In [ ]:
# For Total Season

In [ ]:
matchup_season_full_playoffs=matchup_totals_year_playoffs[(matchup_totals_year_playoffs["Category"]=='Playoffs')]

In [ ]:
matchup_season_full_totals_playoffs=pd.DataFrame(matchup_season_full_playoffs.groupby(by=['PLAYER_NAME',"SEASON"])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_full_totals_playoffs["matchupFieldGoalsPercentage"]=matchup_season_full_totals_playoffs["matchupFieldGoalsMade"]/matchup_season_full_totals_playoffs["matchupFieldGoalsAttempted"]
matchup_season_full_totals_playoffs["matchupThreePointersPercentage"]=matchup_season_full_totals_playoffs["matchupThreePointersMade"]/matchup_season_full_totals_playoffs["matchupThreePointersAttempted"]
matchup_season_full_totals_playoffs["matchupFreeThrowsPercentage"]=matchup_season_full_totals_playoffs["matchupFreeThrowsMade"]/matchup_season_full_totals_playoffs["matchupFreeThrowsAttempted"]
matchup_season_full_totals_playoffs=matchup_season_full_totals_playoffs.fillna(0)

In [ ]:
matchup_season_full_totals_playoffs.head()

,PLAYER_NAME,SEASON,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,shootingFouls,switchesOn,teamPoints,matchupFieldGoalsPercentage,matchupThreePointersPercentage,matchupFreeThrowsPercentage
0,A.J. Lawson,2023-24,0,0,0,16,4,130,60,18,11,6753,0,45,12,12,654.6,143,9,0,694,0.461538,0.266667,0.611111
1,AJ Green,2023-24,0,0,0,29,2,201,86,11,10,13350,0,170,69,9,1186.8,251,4,0,1346,0.427861,0.405882,0.909091
2,AJ Griffin,2023-24,0,0,0,6,3,66,20,0,0,3904,0,38,11,7,347.4,51,0,0,369,0.30303,0.289474,NaN
3,Aaron Gordon,2023-24,0,0,0,250,40,827,448,212,141,55090,0,125,35,105,4635.5,1072,111,0,5785,0.541717,0.28,0.665094
4,Aaron Holiday,2023-24,0,0,0,138,18,432,194,44,41,27321,0,213,83,58,2506.8,512,17,0,2786,0.449074,0.389671,0.931818


In [ ]:
category_full_season_playoffs="matchupFreeThrowsAttempted"

# category=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [ ]:
fig_matchup_season_full_playoffs = px.bar(
  matchup_season_full_totals_playoffs[matchup_season_full_totals_playoffs[category_full_season_playoffs]!=0].sort_values(by=category_full_season_playoffs, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_full_season_playoffs,
  color="PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Player {} For The {} Playoffs</b>".format(category_full_season_playoffs,year_playoffs),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
  height=800, 
  width=1200,
  )
fig_matchup_season_full_playoffs.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season_full_playoffs

In [ ]:
fig_matchup_season_full_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Defender/{}/Player {} For The {} Playoffs".format(save_year,category_full_season_playoffs,year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Metric Plot

In [ ]:
category_full_season_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_full_season_playoffs:
       fig_matchup_season_full_multi_playoffs = px.bar(
       matchup_season_full_totals_playoffs[matchup_season_full_totals_playoffs[category_full_season_playoffs]!=0].sort_values(by=category_full_season_playoffs, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_full_season_playoffs,
       color="PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Player {} For The {} Playoffs</b>".format(category_full_season_playoffs,year),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_full_multi_playoffs.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_full_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/Defender/{}/Player {} For The {} Playoffs".format(save_year,i,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#For All Time 

In [ ]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_season_all_playoffs=matchups_totals[(matchups_totals["Category"]=='Playoffs')]
matchup_season_all_totals_playoffs=pd.DataFrame(matchup_season_all_playoffs.groupby(by=['PLAYER_NAME'])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_all_totals_playoffs["matchupFieldGoalsPercentage"]=matchup_season_all_totals_playoffs["matchupFieldGoalsMade"]/matchup_season_all_totals_playoffs["matchupFieldGoalsAttempted"]
matchup_season_all_totals_playoffs["matchupThreePointersPercentage"]=matchup_season_all_totals_playoffs["matchupThreePointersMade"]/matchup_season_all_totals_playoffs["matchupThreePointersAttempted"]
matchup_season_all_totals_playoffs["matchupFreeThrowsPercentage"]=matchup_season_all_totals_playoffs["matchupFreeThrowsMade"]/matchup_season_all_totals_playoffs["matchupFreeThrowsAttempted"]
matchup_season_all_totals_playoffs=matchup_season_all_totals_playoffs.fillna(0)

In [ ]:
category_all_playoffs="matchupFreeThrowsAttempted"

# category_all_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [ ]:
fig_matchup_all_playoffs = px.bar(
  matchup_season_all_totals_playoffs[matchup_season_all_totals_playoffs[category_all_playoffs]!=0].sort_values(by=category_all_playoffs, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_all_playoffs,
  template='plotly_white',
  title="<b>Playoffs All Time Player {}</b>".format(category_all_playoffs),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
  height=800, 
  width=1200,
  )
fig_matchup_all_playoffs.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_all_playoffs.update_traces(marker_color='#0047ab')
fig_matchup_all_playoffs

In [ ]:
fig_matchup_all_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/All Time Defender/{}/Playoffs All Time Player {}".format(save_year,category_all),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Plots

In [ ]:
category_all_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_all_playoffs:
       fig_matchup_all_multi_playoffs = px.bar(
       matchup_season_all_totals_playoffs[matchup_season_all_totals_playoffs[category_all_playoffs]!=0].sort_values(by=category_all_playoffs, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_all_playoffs,
       template='plotly_white',
       title="<b>All Time Player {}</b>".format(category_all_playoffs),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
       height=800, 
       width=1200,
       )
       fig_matchup_all_multi_playoffs.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_all_multi_playoffs.update_traces(marker_color='#0047ab')
       fig_matchup_all_multi_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Matchups/All Time Defender/{}/Playoffs All Time Player {}".format(save_year,category_all),full_html=False, include_plotlyjs='cdn')